## 유저, 아이템 major judging

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import time
import re

In [2]:
data_dir = '/opt/ml/input/data/train/'

In [3]:
raw_data = pd.read_csv(os.path.join(data_dir, 'train_ratings.csv'))
# item info
writers = pd.read_csv(os.path.join(data_dir, 'writers.tsv'), sep='\t')
director = pd.read_csv(os.path.join(data_dir, 'directors.tsv'), sep='\t')
title = pd.read_csv(os.path.join(data_dir, 'titles.tsv'), sep='\t')
genre = pd.read_csv(os.path.join(data_dir, 'genres.tsv'), sep='\t')
year = pd.read_csv(os.path.join(data_dir, 'years.tsv'), sep='\t')

In [31]:
raw_data.groupby('item').count().describe()

,user,time
count,6807.000000,6807.000000
mean,757.230939,757.230939
std,1682.973090,1682.973090
min,27.000000,27.000000
25%,90.000000,90.000000
50%,197.000000,197.000000
75%,610.500000,610.500000
max,19699.000000,19699.000000


-----------------------

### 유저별 메이저 영화의 비율 계산

In [43]:
def get_item_count(data):
    itemcount = data.groupby('item').count()
    return {itemcount.iloc[i].name : itemcount.iloc[i]['user'] for i in range(itemcount.shape[0])}

item_count = get_item_count(raw_data)

In [53]:
users_new = list(raw_data['user'])
items_new = [item_count[i] for i in raw_data['item']]
count_df = pd.DataFrame(zip(users_new,items_new), columns=['user','item_count'])
items_bool = list(count_df['item_count'] >=2500)
bool_df = pd.DataFrame(zip(users_new,items_bool), columns=['user','major'])

In [ ]:
ratio_df = bool_df.groupby('user').agg(['sum', 'count'])['major']
temp = {ratio_df.iloc[i].name : round(ratio_df.iloc[i]['sum']/ratio_df.iloc[i]['count'],2)  for i in range(ratio_df.shape[0])}

In [77]:
ratios = pd.DataFrame(zip(temp.keys(),temp.values()), columns=['user','ratio'])

In [80]:
ratios['ratio'].describe()

count    31360.000000
mean         0.598029
std          0.176820
min          0.000000
25%          0.470000
50%          0.610000
75%          0.730000
max          1.000000
Name: ratio, dtype: float64